In [ ]:
import os
import time
import emoji
import random
import datetime
import pyperclip
import traceback
import numpy as np
import pandas as pd
from selenium import webdriver
from config import CHROME_DATA_PATH
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import UnexpectedAlertPresentException

import warnings
warnings.filterwarnings("ignore")

# Record the start time
start_time = time.time()

# Read the both Excel data at the beginning
excel_data = pd.read_excel('Recipients data.xlsx')
new_excel_data = pd.read_excel('new_lead.xlsx')

# Path to the Excel file where new numbers will be saved
new_numbers_excel_file = 'new_numbers.xlsx'

# Clean the 'Contact' column in both DataFrames to remove spaces and special characters
clean_contact = lambda x: ''.join(filter(str.isdigit, str(x)))
excel_data['Contact'] = excel_data['Contact'].apply(clean_contact)
new_excel_data['Contact'] = new_excel_data['Contact'].apply(clean_contact)

# Create a list of existing numbers
existing_numbers = set(excel_data['Contact'].values)

# Filter out the new numbers that do not exist in the old DataFrame
new_numbers = new_excel_data[~new_excel_data['Contact'].isin(existing_numbers)]

# If there are new numbers, add them to the old DataFrame and save them to 'new_numbers.xlsx'
if not new_numbers.empty:
    new_numbers.to_excel(new_numbers_excel_file, index=False)
    print(f"New numbers added to 'Recipients data.xlsx' and saved in 'new_numbers.xlsx'.")
else:
    print("No new numbers found in the 'new_lead.xlsx' file.")

# Rest of the Selenium code
count = 0

# Initializing the driver to automate the Chrome browser
os.system("taskkill /im chrome.exe /f")
options = webdriver.ChromeOptions()
options.add_argument(CHROME_DATA_PATH)
driver = webdriver.Chrome( options=options)

url = f'https://web.whatsapp.com/send?phone={123456789101}'
driver.get(url)
time.sleep(10)

def custom_split(text):
    paragraphs = text.split('\n\n')
    result = []

    # Iterate through paragraphs
    for i, paragraph in enumerate(paragraphs):
        # Split each paragraph by '\n'
        lines = paragraph.split('\n')

        # Extend result with lines and add a newline if it's not the last paragraph
        result.extend(lines)
        if i < len(paragraphs) - 1:
            result.append('')

    return result

# Function to open a chat window and send a message
def open_chat_and_send_message(contact, message):
    url = f'https://web.whatsapp.com/send?phone={contact}'
    driver.get(url)
    time.sleep(round(np.random.uniform(20, 30), 2))  # Wait for search results to load
    
    try:
        # Wait for the attachment box element to be present
        attachment_box = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@title="Attach"]')))
        attachment_box.click()
        time.sleep(5)
#         time.sleep(round(np.random.uniform(5, 10), 2))

        # Wait for the document upload dialog to appear
        document_box = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, "//input[@accept='*']")))
        document_box.send_keys(r"C:\Users\Ehsan\AI Intern\Empire Suites.pdf")
        time.sleep(2)
#         time.sleep(round(np.random.uniform(5, 10), 2))

        # Wait for the chat window to load
        chat_input = WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//div[@title="Type a message"]')))
        chat_input.clear()
        # Send date and time
        chat_input.send_keys(f'{datetime.datetime.now().strftime("%d-%B-%Y" + "  " + "(" + "%I:%M %p" + ")")}')
        chat_input.send_keys(Keys.SHIFT, Keys.RETURN)  # Press Shift+Enter to create a new line
        chat_input.send_keys(Keys.SHIFT, Keys.RETURN)  # Press Shift+Enter to create a new line
#         time.sleep(round(np.random.uniform(5, 10), 2))

        # Send recipient name
        chat_input.send_keys(f'Dear {recipient_name},')
        chat_input.send_keys(Keys.SHIFT, Keys.RETURN)  # Press Shift+Enter to create a new line
#         time.sleep(round(np.random.uniform(5, 10), 2))
        
        # Use the custom_split function to format the message
        formatted_message = custom_split(message)

        # Copy the formatted message to clipboard using pyperclip
        pyperclip.copy('\n'.join(formatted_message))

        # Paste the message into the chat
        chat_input.send_keys(Keys.CONTROL, 'v')
#         time.sleep(round(np.random.uniform(20, 40), 2))
        chat_input.send_keys(Keys.RETURN)  # Send the complete message
        print(f'{count}- Message sent to {contact} successfully.')
    
    except NoSuchElementException as e:
        print(f"Element not found: {str(e)}")
        
    except TimeoutException as e:
        print(f"Timed out waiting for element: {str(e)}")
        
    except UnexpectedAlertPresentException as alert_exception:
        # Handle unexpected alerts by dismissing them
        print(f'Alert appeared unexpectedly: {str(alert_exception)}')
        try:
            driver.switch_to.alert.dismiss()
        except:
            pass  # Alert might not be present
    except Exception as e:
        print(f"Failed to send message to {contact}: {str(e)}")
        traceback.print_exc()  # Add this line to print the full traceback

for index, row in excel_data.iterrows():
    time.sleep(2)
    try:
        contact = str(row['Contact'])
        recipient_name = row['Names']

        count += 1
        
        message = emoji.emojize(custom_message)
        # Open the chat window and send the message
        open_chat_and_send_message(contact, message)
        time.sleep(5)
    except Exception as e:
        print(f"Failed to send message to {contact}: {str(e)}")
        traceback.print_exc()  # Add this line to print the full traceback

driver.quit()

# Record the end time
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time
print(f'The script executed successfully {count} times.')
print(f'Total execution time: {execution_time} seconds')